In [80]:
import torch
import data_loader
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
import torch.optim as optim

In [50]:
trainx, devx, testx, trainy, devy, testy = data_loader.load_all_classic_random_split(flatten=False)
trainx.shape, devx.shape, testx.shape, trainy.shape, devy.shape, testy.shape

Processing albert
Processing canon_12_5
Processing chen_12_7
Processing daniel
Processing haobin_11_22
Processing isa_12_5
Processing janet
Processing joanne
Processing jq_12_6
Processing kelly_11_7
Processing kevin_11_7
Processing ruocheng
Processing russell_11_20_stand
Processing russell_11_7
Processing russell_random_12_7
Processing solomon
Processing wenzhou_12_5
Processing yiheng_11_30
Processing yiheng_12_5
Processing yongxu_11_30
Processing zhaoye_12_7
Splitting out test set
Splitting out dev and train set


((8320, 100, 3), (1040, 100, 3), (1041, 100, 3), (8320,), (1040,), (1041,))

In [158]:
trainset = [(trainx[i].T, trainy[i]) for i in range(trainy.shape[0])]
# trainset
testset = [(testx[i].T, testy[i]) for i in range(testy.shape[0])]

In [191]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=1, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=True)


In [154]:
for i, (x,y) in enumerate(trainloader):
    print(i, x.float().shape)
    raise

0 torch.Size([1, 3, 100])


RuntimeError: No active exception to reraise

In [176]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv = nn.Conv1d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)
        self.fc = nn.Linear(800, 500)
        self.out = nn.Linear(500, 26)

    def forward(self, x):
        # x = self.pool(F.relu(self.conv1(x)))
        # x = self.pool(F.relu(self.conv2(x)))
        # x = x.view(-1, 16 * 5 * 5)
        # x = F.relu(self.fc1(x))
        # x = F.relu(self.fc2(x))
        # x = self.fc3(x)
        x = self.pool(F.relu(self.conv(x)))
        x = x.view(-1, 800)
        x = F.relu(self.fc(x))
        x = self.out(x)
        return x
net = Net()

In [193]:

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.00001, momentum=0.9)

for epoch in range(4):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        # print(inputs.shape)
        # print(labels)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs.float())
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 1.618
[1,  4000] loss: 1.589
[1,  6000] loss: 1.662
[1,  8000] loss: 1.641
[2,  2000] loss: 1.483
[2,  4000] loss: 1.438
[2,  6000] loss: 1.428
[2,  8000] loss: 1.406
[3,  2000] loss: 1.281
[3,  4000] loss: 1.308
[3,  6000] loss: 1.284
[3,  8000] loss: 1.335
[4,  2000] loss: 1.099
[4,  4000] loss: 1.208
[4,  6000] loss: 1.184
[4,  8000] loss: 1.192
Finished Training


In [196]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        x, y = data
        outputs = net(x.float())
        _, predicted = torch.max(outputs.data, 1)
        if predicted.data == y.data:
            correct += 1
        total+= 1
print(correct / total)

0.6003842459173871
